In [3]:
%pip install html5lib
%pip install webdriver-manager
%pip install selenium


   ---------------------------------------- 0.0/112.2 kB ? eta -:--:--
   --- ------------------------------------ 10.2/112.2 kB ? eta -:--:--
   --- ------------------------------------ 10.2/112.2 kB ? eta -:--:--
   -------------- ------------------------ 41.0/112.2 kB 279.3 kB/s eta 0:00:01
   -------------------------------- ------ 92.2/112.2 kB 525.1 kB/s eta 0:00:01
   -------------------------------------- 112.2/112.2 kB 502.1 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re

## 房价信息爬取

In [31]:
# 设置WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 目标URL
base_url = "https://esf.fang.com/house-a010-b01515/"

In [33]:
# 存储所有房源信息
all_properties = []

try:
    # 遍历前20页
    for page in range(1, 21):
        if page == 1:
            url = base_url
        else:
            url = f"{base_url}i3{page}/"
        
        print(f"正在爬取第{page}页: {url}")
        driver.get(url)
        
        # 等待页面加载完成
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "shop_list"))
        )
        
        # 获取所有房源元素
        property_list = driver.find_elements(By.CLASS_NAME, "shop_list dl")
        print(f"找到{len(property_list)}个房源")
        
        # 提取每个房源的信息
        for property_item in property_list:
            try:
                # 初始化字典存储房源信息
                property_info = {}
                
                # 提取标题和链接
                title_element = property_item.find_element(By.CLASS_NAME, "tit_shop")
                property_info['标题'] = title_element.text
                # property_info['链接'] = title_element.get_attribute('href')
                
                # 提取价格信息
                try:
                    price_element = property_item.find_element(By.CLASS_NAME, "price_right")
                    price_text = price_element.text
                    property_info['总价'] = price_text.split('万')[0] + '万' if '万' in price_text else price_text
                    
                    # 尝试提取单价
                    try:
                        unit_price_element = price_element.find_element(By.TAG_NAME, "span")
                        property_info['单价'] = unit_price_element.text
                    except NoSuchElementException:
                        property_info['单价'] = "未知"
                except NoSuchElementException:
                    property_info['总价'] = "未知"
                    property_info['单价'] = "未知"
                
                # 提取房屋基本信息
                try:
                    tel_shop = property_item.find_element(By.CLASS_NAME, "tel_shop")
                    details = tel_shop.text.split('|')
                    
                    # 解析户型、面积、朝向等信息
                    for detail in details:
                        detail = detail.strip()
                        if '室' in detail and '厅' in detail:
                            property_info['户型'] = detail
                        elif '㎡' in detail:
                            property_info['面积'] = detail
                        elif '向' in detail:
                            property_info['朝向'] = detail
                        elif '层' in detail:
                            property_info['楼层'] = detail
                        elif '年建' in detail:
                            property_info['建成年份'] = detail
                except NoSuchElementException:
                    pass
                
                # 提取小区名称和地址
                try:
                    add_shop = property_item.find_element(By.CLASS_NAME, "add_shop")
                    address_parts = add_shop.text.split('\n')
                    if len(address_parts) >= 1:
                        property_info['小区名称'] = address_parts[0]
                    if len(address_parts) >= 2:
                        property_info['地址'] = address_parts[1]
                except NoSuchElementException:
                    pass
                
                # 提取特色标签
                try:
                    label_elements = property_item.find_elements(By.CLASS_NAME, "label")
                    labels = [label.text for label in label_elements if label.text]
                    property_info['特色标签'] = '|'.join(labels)
                except NoSuchElementException:
                    property_info['特色标签'] = ""
                
                # 提取经纪人信息
                try:
                    people_element = property_item.find_element(By.CLASS_NAME, "people_name")
                    property_info['经纪人'] = people_element.text
                except NoSuchElementException:
                    property_info['经纪人'] = "未知"
                
                # 提取交通信息
                try:
                    # 尝试查找交通信息，可能需要进一步解析
                    traffic_info = property_item.find_element(By.XPATH, ".//*[contains(text(), '距')]")
                    property_info['交通信息'] = traffic_info.text
                except NoSuchElementException:
                    property_info['交通信息'] = "未知"
                
                # 将房源信息添加到列表
                all_properties.append(property_info)
                # print(f"已提取房源: {property_info['标题']}")
                
            except Exception as e:
                print(f"提取房源信息时出错: {e}")
                continue
        
        # 等待一段时间再请求下一页，避免过于频繁
        time.sleep(2)
        
except Exception as e:
    print(f"爬取过程中出错: {e}")
finally:
    driver.quit()

# 将数据转换为DataFrame
df = pd.DataFrame(all_properties)

正在爬取第1页: https://esf.fang.com/house-a010-b01515/
找到60个房源
正在爬取第2页: https://esf.fang.com/house-a010-b01515/i32/
找到60个房源
正在爬取第3页: https://esf.fang.com/house-a010-b01515/i33/
找到60个房源
正在爬取第4页: https://esf.fang.com/house-a010-b01515/i34/
找到60个房源
正在爬取第5页: https://esf.fang.com/house-a010-b01515/i35/
找到60个房源
正在爬取第6页: https://esf.fang.com/house-a010-b01515/i36/
找到60个房源
正在爬取第7页: https://esf.fang.com/house-a010-b01515/i37/
找到60个房源
正在爬取第8页: https://esf.fang.com/house-a010-b01515/i38/
找到60个房源
正在爬取第9页: https://esf.fang.com/house-a010-b01515/i39/
找到60个房源
正在爬取第10页: https://esf.fang.com/house-a010-b01515/i310/
找到60个房源
正在爬取第11页: https://esf.fang.com/house-a010-b01515/i311/
找到60个房源
正在爬取第12页: https://esf.fang.com/house-a010-b01515/i312/
找到60个房源
正在爬取第13页: https://esf.fang.com/house-a010-b01515/i313/
找到60个房源
正在爬取第14页: https://esf.fang.com/house-a010-b01515/i314/
找到60个房源
正在爬取第15页: https://esf.fang.com/house-a010-b01515/i315/
找到60个房源
正在爬取第16页: https://esf.fang.com/house-a010-b01515/i316/
找到60个房源
正在爬取第17页: http

In [36]:
# 保存到CSV文件
df.to_csv('Housing_Price_YiZhuang.csv', index=False, encoding='utf-8-sig')
print(f"爬取完成，共获取{len(all_properties)}条房源信息，已保存到fangtianxia_properties.csv")

爬取完成，共获取1200条房源信息，已保存到fangtianxia_properties.csv


## 租价信息爬取

In [59]:
# 设置WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 目标URL
base_url_rent = "https://zu.fang.com/house-a010-b01515/"

In [61]:
# 存储所有房源信息
all_properties = []

try:
    # 遍历前20页
    for page in range(1, 21):
        if page == 1:
            url = base_url_rent
        else:
            url = f"{base_url_rent}i3{page}/"
        
        print(f"正在爬取第{page}页: {url}")
        driver.get(url)
        
        # 等待页面加载完成
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CLASS_NAME, "houseList"))
        )
        
        # 获取所有房源元素 - 根据图片2中的HTML结构
        property_list = driver.find_elements(By.CSS_SELECTOR, ".houseList > .list")
        print(f"找到{len(property_list)}个房源")
        
        # 提取每个房源的信息
        for index, property_item in enumerate(property_list):
            try:
                # 初始化字典存储房源信息
                property_info = {}
                property_info['页码'] = page
                
                # 提取标题和链接 - 根据图片2中的HTML结构
                try:
                    title_element = property_item.find_element(By.CSS_SELECTOR, ".title a")
                    property_info['标题'] = title_element.text.strip()
                    property_info['链接'] = title_element.get_attribute('href')
                except NoSuchElementException:
                    print(f"第{page}页第{index+1}个房源缺少标题信息")
                    continue
                
                # 提取价格信息 - 根据图片1中的信息
                try:
                    # 尝试查找价格元素，可能类名为"money"或其他
                    price_element = property_item.find_element(By.CSS_SELECTOR, ".money")
                    property_info['月租金'] = price_element.text.strip()
                except NoSuchElementException:
                    # 如果找不到特定类名，尝试通过文本内容查找
                    try:
                        price_text = property_item.find_element(By.XPATH, ".//*[contains(text(), '元/月')]")
                        property_info['月租金'] = price_text.text.strip()
                    except:
                        property_info['月租金'] = "未知"
                
                # 提取房屋基本信息 - 根据图片2中的HTML结构
                try:
                    info_element = property_item.find_element(By.CLASS_NAME, "font15")
                    info_text = info_element.text
                    
                    # 解析租赁类型、户型、面积、朝向等信息
                    parts = info_text.split('|')
                    for part in parts:
                        part = part.strip()
                        if '整租' in part or '合租' in part:
                            property_info['租赁类型'] = part
                        elif '室' in part and '厅' in part:
                            property_info['户型'] = part
                        elif '㎡' in part:
                            property_info['面积'] = part
                        elif '朝' in part:
                            property_info['朝向'] = part
                except NoSuchElementException:
                    pass
                
                # 提取位置信息 - 根据图片1中的信息
                try:
                    # 尝试查找位置元素，可能类名为"gray6"或其他
                    location_element = property_item.find_element(By.CLASS_NAME, "gray6")
                    location_text = location_element.text.strip()
                    
                    # 解析区域-商圈-小区信息
                    if '-' in location_text:
                        parts = location_text.split('-')
                        if len(parts) >= 3:
                            property_info['区域'] = parts[0]
                            property_info['商圈'] = parts[1]
                            property_info['小区'] = parts[2]
                        elif len(parts) == 2:
                            property_info['区域'] = parts[0]
                            property_info['商圈'] = parts[1]
                    else:
                        property_info['位置'] = location_text
                except NoSuchElementException:
                    pass
                
                # 提取交通信息 - 根据图片1中的信息
                try:
                    # 尝试查找包含"距"字的元素
                    traffic_element = property_item.find_element(By.XPATH, ".//*[contains(text(), '距')]")
                    property_info['交通信息'] = traffic_element.text.strip()
                except NoSuchElementException:
                    property_info['交通信息'] = "未知"
                
                # 提取特色标签 - 根据图片2中的HTML结构
                try:
                    # 尝试查找标签元素，可能类名为"note"或其他
                    tag_elements = property_item.find_elements(By.CLASS_NAME, "note")
                    tags = [tag.text for tag in tag_elements if tag.text.strip()]
                    property_info['特色标签'] = '|'.join(tags)
                except NoSuchElementException:
                    property_info['特色标签'] = ""
                
                # 将房源信息添加到列表
                all_properties.append(property_info)
                print(f"已提取第{page}页第{index+1}个房源: {property_info['标题']}")
                
            except Exception as e:
                print(f"提取第{page}页第{index+1}个房源信息时出错: {e}")
                continue
        
        # 等待一段时间再请求下一页，避免过于频繁
        time.sleep(2)
        
except Exception as e:
    print(f"爬取过程中出错: {e}")
finally:
    driver.quit()

# 将数据转换为DataFrame
df = pd.DataFrame(all_properties)


正在爬取第1页: https://zu.fang.com/house-a010-b01515/
找到60个房源
已提取第1页第1个房源: 5室2厅DearVilla
已提取第1页第2个房源: 南北通透国锐金嵿5室2厅
已提取第1页第3个房源: 南北通透国锐金嵿4室2厅精装修
已提取第1页第4个房源: 4室2厅枫丹壹号2期
已提取第1页第5个房源: 10室3厅林肯公园2期A区
已提取第1页第6个房源: 君安国际公寓3室2厅精装修
已提取第1页第7个房源: 南北通透国锐金嵿4室2厅豪华装修
已提取第1页第8个房源: 南北通透国锐金嵿5室2厅精装修
已提取第1页第9个房源: 5室2厅金地格林小镇
已提取第1页第10个房源: 南北通透亦庄金茂府3室2厅精装修
已提取第1页第11个房源: 国锐金嵿2室1厅精装修
已提取第1页第12个房源: 南北通透国锐金嵿5室2厅精装修
已提取第1页第13个房源: 5室2厅一栋洋房二区
已提取第1页第14个房源: 林肯时代3室2厅精装修
已提取第1页第15个房源: 南北通透国锐金嵿5室2厅
已提取第1页第16个房源: 5室2厅林肯公园2期B区
已提取第1页第17个房源: 南北通透国锐金嵿4室2厅豪华装修
已提取第1页第18个房源: 林肯时代3室2厅精装修
已提取第1页第19个房源: 林肯时代3室1厅豪华装修
已提取第1页第20个房源: 3室2厅金地格林小镇
已提取第1页第21个房源: 4室2厅一栋洋房五区
已提取第1页第22个房源: 3室2厅金地格林小镇
已提取第1页第23个房源: 3室2厅金地格林小镇
已提取第1页第24个房源: 林肯时代3室2厅精装修
已提取第1页第25个房源: 林肯时代3室2厅精装修
已提取第1页第26个房源: 国锐金嵿2室1厅精装修
已提取第1页第27个房源: 林肯时代3室1厅豪华装修
已提取第1页第28个房源: 林肯时代3室2厅精装修
已提取第1页第29个房源: 林肯时代3室2厅精装修
已提取第1页第30个房源: 4室2厅金地格林小镇
已提取第1页第31个房源: 国锐金嵿2室1厅精装修
已提取第1页第32个房源: 国锐金嵿4室2厅豪华装修
已提取第1页第33个房源: 君安国际公寓3室2厅精装修
已提取第1页第34个房源: 林肯公园C区住宅2室2厅精装修
已提取第1页第35个房源: 3室2厅中信新城东区
已提取第1页第

In [63]:
# 保存到CSV文件
df.to_csv('Housing_Rent_YiZhuang.csv', index=False, encoding='utf-8-sig')
print(f"爬取完成，共获取{len(all_properties)}条房源信息，已保存到fangtianxia_rental_properties.csv")

爬取完成，共获取1200条房源信息，已保存到fangtianxia_rental_properties.csv
